In [ ]:
!pip install sentencepiece
!pip install nltk
! pip install -U transformers
! pip install -U simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch

from transformers import AutoTokenizer, AutoModel

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import operator 

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터셋 읽어오기
data = pd.read_csv('/content/drive/MyDrive/빅데이터처리 개인 폴더/steam_data(to 40000).csv',encoding='UTF-8')
text_data = data['Game Description'].tolist()

In [ ]:
data.head()

,Unnamed: 0.1,Unnamed: 0,Overall Reviews,Recent Reviews,Most Helpful Reviews,Currently popular,Game Description
0,0,0,Mostly Negative,NaN,"<div id=""game_area_metalink"">Not yet reviewed....","['Not Recommended, \n\t\t\t\t\tAt first glance...",Galactic Bowling is an exaggerated and stylize...
1,0,0,Very Positive,NaN,NaN,"[""Recommended, \n\t\t\t\t\tOne Finger Death Pu...",THE LAW!!Looks to be a showdown atop a train. ...
2,0,0,NaN,NaN,NaN,[],Jolt Project:The army now has a new robotics p...
3,0,0,No user reviews,NaN,NaN,"['Recommended, \nIntroductionWhile 2D platform...","In bizarre lands, play as a viscous glob of wa..."
4,0,0,Very Positive,NaN,NaN,"['Not Recommended, \n\t\t\t\t\tThere’s a reaso...",ABOUT THE GAMEPlay as a hacker who has arrange...


In [ ]:
# kaggle data에서 수집한 장르 종류 (쥬피터에서 돌려서 긁어옴)

genres = ['Casual',
 'Indie',
 'Sports',
 'Action',
 'Adventure',
 'Strategy',
 'RPG',
 'Simulation',
 'Early Access',
 'Racing',
 'Massively Multiplayer',
 'Utilities',
 'Education',
 'Sexual Content',
 'Nudity',
 'Violent',
 'Gore',
 'Web Publishing',
 'Animation & Modeling',
 'Design & Illustration',
 'Free to Play',
 'Software Training',
 'Game Development',
 'Photo Editing',
 'Audio Production',
 'Video Production',
 'Accounting',
 'Movie',
 'Documentary',
 'Episodic',
 'Short',
 'Tutorial',
 '360 Video',
 'Survival']

# 소문자 처
genres_lower = [g.lower() for g in genres]
genres_lower

['casual',
 'indie',
 'sports',
 'action',
 'adventure',
 'strategy',
 'rpg',
 'simulation',
 'early access',
 'racing',
 'massively multiplayer',
 'utilities',
 'education',
 'sexual content',
 'nudity',
 'violent',
 'gore',
 'web publishing',
 'animation & modeling',
 'design & illustration',
 'free to play',
 'software training',
 'game development',
 'photo editing',
 'audio production',
 'video production',
 'accounting',
 'movie',
 'documentary',
 'episodic',
 'short',
 'tutorial',
 '360 video',
 'survival']

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
def remove_stopwords(sentence):
    # 불용어 제거 
    stopwords = nltk.corpus.stopwords.words('english')
    stop_words = set(stopwords)
    words = word_tokenize(sentence)

    filtered_words = [word for word in words if word.casefold() not in stop_words]
    # 다시 문장으로 만들기 
    filtered_sentence = ' '.join(filtered_words)
    return filtered_sentence

def preprocess_text(text):
    # 특수 문자 제거
    text = re.sub(r'[^\w\s]', ' ', text)

    return text


def extract_keywords(text, top_k=30):

    text = remove_stopwords(text)
    text = preprocess_text(text)

    print("After Proprocessing: \n",text, '\n')

    # XLNet tokenizer 및 모델 로드
    tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
    
    model = AutoModel.from_pretrained("xlnet-base-cased")
    model.to(device)
    '''
    # 입력 텍스트 토큰화
    tokens = tokenizer.tokenize(text)

    input_ids = tokenizer.encode(text, return_tensors='pt') # XLNet에 맞게 속성 수정 
    input_ids = input_ids.to(device)
    '''
    tokens = tokenizer.tokenize(text)
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # XLNet 모델 실행
    with torch.no_grad():
        outputs = model(input_ids=input_ids,attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state 
    '''
    # XLNet 모델 실행
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        last_hidden_state = outputs.last_hidden_state
    '''
    # 키워드 추출
    # last_hidden_state[0].mean(dim=1) : 각 문장의 임베딩 벡터들을 평균화한 하나의 벡터 (해당 문장의 전체 문맥을 나타내는 벡터)
    #  ~.indices:  가장 큰 top_k개의 값들의 인덱스를 나타내는 텐서 (가장 중요한 토큰들의 위치를 확인할 수 있음)
    keyword_ids = torch.topk(last_hidden_state[0].mean(dim=1), top_k).indices.squeeze()
    keywords = [tokens[idx] for idx in set(keyword_ids)]

    return keywords


#######################################################################
# 예시 문장
index = int(input())
text = text_data[index]#'There is a cat. this cat can walk and grab guns. cat will revenge about the abusing. You can choose background photo of this game and type of gun among the 54 guns. and also you can customize cat character using photos in your computer. '# text_data[0]
if 'nan' in text:
  "NAN"
print('TEXT >>> \n"',text,'"\n\n')

# 키워드 보기
final_key = []

text_split, i = text.split(' '), 0

while i < len(text_split):
  try:
    n = int(text_split[i][0])
    if len(text_split[i]) == 4: # 년도일 가능성 높음
      final_key.append(text_split[i])
      i += 1
    elif len(text_split[i+1]) > 2:
      final_key.append(text_split[i].lower() +' '+ text_split[i+1].lower())
      i += 2
  except:
    i += 1



# 추출하기 & 단어만 골라내기
top_k = 30
while(True): # 만약 top_k개 뽑아낼 수 없는 경우 top_k를 줄여가며 시도
  if text == 'nan': break
  try: 
    keywords_extracted = set(extract_keywords(text, top_k))
    break
  except:
     top_k -= 5


# XLNet 토크나이'▁' 때문에 명사 처럼 여겨지는 경우가 있어 처리
keywords_extracted_copy = keywords_extracted.copy()
for t in keywords_extracted_copy :
  if t == '▁':
    keywords_extracted.remove(t)
  elif t[0] == '▁':
    keywords_extracted.remove(t)
    t = t[1:]
    keywords_extracted.add(t)

# 품사 태깅
tokens_pos = nltk.pos_tag(keywords_extracted)


# 명사, 형용사만 골라내기
only_nORj = [] 
for word, pos in tokens_pos:
    if 'NN' in pos  or 'JJ' in pos:
      w_lower = word.lower() # 소문자로 통일
      for gl in genres_lower: # 장르 관련 키워드면 무조건 포함
        if gl in w_lower:
          final_key.append(w_lower)
      else: only_nORj.append(w_lower)

only_nORj = set(only_nORj)
# 여러 조건들에 맞춰 해당하는 것만 뽑아내기
for w in only_nORj:
  if len(w) <= 3: continue # 2글자 이하 제거
  if w[-1] == 's' : # 끝에 s가 붙는 경우 s제거
    w = w[:-1]
  if ('game' in w) or ('thing' in w) or (w =='play') or w == 'fea': continue
  append_bool = True
  '''
  for key in final_key:
    if w in key:
      append_bool = False
      break
  '''
  if append_bool: final_key.append(w)

print('\n\nKEY WORDS : ',set(final_key))

103
TEXT >>> 
" The most secure prison facility in the countryis under attack ...The city mayor called personally and asked you to take care of this problem.You are Jake, ex-military and a member of Team . Mission Ammunition is a modern action game with a lot of justified pixel violence and the clever platforming.                                                   Over 30 levels of action Ultra-sharp and detailed pixel art Super-tough boss fights Very reasonable number of powerful guns Great accompanying music Top secret locations Spectacular explosions Pure old-school awesomeness "


After Proprocessing: 
 secure prison facility countryis attack     city mayor called personally asked take care problem You Jake   ex military member Team   Mission Ammunition modern action game lot justified pixel violence clever platforming   30 levels action Ultra sharp detailed pixel art Super tough boss fights reasonable number powerful guns Great accompanying music Top secret locations Spectacular ex

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.




KEY WORDS :  {'problem', '30 levels', 'tough', 'mayor', 'nes', 'member', 'military', 'action', 'sharp', 'city', 'pure', 'ultra', 'school', 'awesome', 'take', 'modern', 'cular', 'spec', 'care', 'pixel', 'violence'}
